# Assignment 1: Starter package
 Aleksei Tiulpin, PhD

 This notebook is a starter package to process the OAI data, and get you started with the assignment.

## Basic imports

In [1]:
import pandas as pd
import numpy as np

# OAI data loading

Fix the path to OAI files in the code below.

In [2]:
enrolees = pd.read_csv("Enrollees.txt", dtype={"ID": str, "SIDE": int}, sep="|")

# Patient level information
clinical = pd.read_csv("AllClinical00.txt", dtype={"ID": str, "SIDE": int}, sep="|")
quant_readings_00 = pd.read_csv("kxr_qjsw_duryea00.txt", dtype={"ID": str, "SIDE": int}, sep="|")
quant_readings_05 = pd.read_csv("kxr_qjsw_duryea05.txt", dtype={"ID": str, "SIDE": int, "side": int}, sep="|")
xr_gradings = pd.read_csv("kxr_sq_bu00.txt", dtype={"ID": str, "SIDE": int}, sep="|")
fnih = pd.read_csv("Clinical_FNIH.txt", dtype={"ID": str, "SIDE": str}, sep="|")

# Building a dataset with verified clean target for both knees

In [3]:
enrolees = enrolees[["ID", "P02SEX"]]

# We should pull data from BOTH KNEES at teh same time
clinical = clinical[["ID", "V00AGE", "P01BP30", "P01FAMHR", "P01FAMKR", "P01INJR", 
                     "P01INJL", "P01KPACDCV", "V00WOMKPL", "V00WOMADLL", "V00WOMKPR", "V00WOMADLR"]]

clinical = pd.merge(clinical, enrolees, on="ID")

oarsi_grades = xr_gradings[["ID", "SIDE", "V00XRJSL", "V00XRJSM", "V00XROSFL", "V00XROSFM", "V00XROSTL", "V00XROSTM"]].dropna()

# Quantitative readings
# Cleaning the baseline data
quant_readings_00.drop_duplicates(subset=['ID', 'SIDE'], inplace=True)
quant_readings_00 = quant_readings_00[(quant_readings_00['V00NOLJSWX'].astype(float) + quant_readings_00['V00NOMJSWX'].astype(float)) == 0]

# Building the full 48 months dataset and cleaning it too
quant_readings_05.rename({"side":"SIDE"}, axis=1, inplace=True)
quant_readings_05 = quant_readings_05[(quant_readings_05['V05NOLJSWX'].astype(float) + quant_readings_05['V05NOMJSWX'].astype(float)) == 0]

# Merging the baseline and 48 months data into 1 dataframe
ds_quant = pd.merge(quant_readings_00[["ID", "SIDE", "V00JSW250"]], quant_readings_05[["ID", "SIDE", "V05JSW250"]], on=("ID", "SIDE"))
ds_quant.drop_duplicates(subset=['ID', 'SIDE'], inplace=True)
ds_quant.dropna(inplace=True)

# We have to make sure that the measurements are accurate. 
# Joint space cannot increase. At least not yet :-)
# Cleaning the data to ensure the correct target definition
# Poorly defined target = bad results
ds_quant = ds_quant[ds_quant.V00JSW250 - ds_quant.V05JSW250 > 0]

# Creating a new variable for progression.
fnih_two_knees = ds_quant.assign(PROG_JSW=ds_quant.V00JSW250 - ds_quant.V05JSW250 >= 0.7)

## Merging the data

In [4]:
## Building a dataset for patients in FNIH
fnih_two_knees = fnih_two_knees[fnih_two_knees.ID.isin(fnih.ID)]
oai_fnih = pd.merge(fnih_two_knees, oarsi_grades, on=("ID", "SIDE"))
# Note that there exists another solution to building the model at a knee level 
oai_fnih = pd.merge(oai_fnih, clinical, on="ID")
oai_fnih

,ID,SIDE,V00JSW250,V05JSW250,PROG_JSW,V00XRJSL,V00XRJSM,V00XROSFL,V00XROSFM,V00XROSTL,...,P01FAMHR,P01FAMKR,P01INJR,P01INJL,P01KPACDCV,V00WOMKPL,V00WOMADLL,V00WOMKPR,V00WOMADLR,P02SEX
0,9002116,1,6.20,5.40,True,0.0,1.0,0.0,1.0,0.0,...,0: No,1: Yes,0: No,0: No,30.0,12.0,40.0,0.0,0.0,1: Male
1,9002116,2,4.70,2.90,True,0.0,2.0,1.0,0.0,1.0,...,0: No,1: Yes,0: No,0: No,30.0,12.0,40.0,0.0,0.0,1: Male
2,9002430,1,6.12,5.67,False,0.0,0.0,1.0,1.0,0.0,...,0: No,0: No,1: Yes,1: Yes,0.0,2.0,9.0,2.0,9.0,1: Male
3,9002430,2,4.54,3.76,True,0.0,2.0,2.0,3.0,0.0,...,0: No,0: No,1: Yes,1: Yes,0.0,2.0,9.0,2.0,9.0,1: Male
4,9002817,1,4.31,4.21,False,0.0,2.0,0.0,2.0,1.0,...,0: No,0: No,0: No,0: No,0.0,0.0,0.0,0.0,0.0,2: Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,9996098,2,6.01,5.63,False,0.0,1.0,0.0,1.0,0.0,...,0: No,0: No,0: No,0: No,20.0,4.0,13.0,3.0,4.0,1: Male
935,9997381,1,4.10,3.43,False,0.0,1.0,0.0,0.0,0.0,...,0: No,0: No,0: No,0: No,0.0,0.0,0.0,0.0,2.0,2: Female
936,9997381,1,4.10,3.43,False,0.0,1.0,0.0,0.0,0.0,...,0: No,0: No,0: No,0: No,0.0,0.0,0.0,0.0,2.0,2: Female
937,9997381,2,4.30,4.27,False,0.0,0.0,0.0,0.0,0.0,...,0: No,0: No,0: No,0: No,0.0,0.0,0.0,0.0,2.0,2: Female


In [ ]:
oai_fnih.drop("V05JSW250", axis=1, inplace=True)
oai_fnih.dropna(inplace=True)
columns_list = oai_fnih.columns.tolist()
print(columns_list)

In [ ]:
oai_fnih.info()

In [298]:
from sklearn.preprocessing import LabelEncoder

features = [ 'V00JSW250', 'V00XRJSL', 'V00XRJSM', 'V00XROSFL', 'V00XROSFM', 'V00XROSTL', 
            'V00XROSTM', 'V00AGE', 'P01BP30', 'P01FAMHR', 'P01FAMKR', 'P01INJR', 'P01INJL', 'P01KPACDCV', 'V00WOMKPL', 
            'V00WOMADLL', 'V00WOMKPR', 'V00WOMADLR', 'P02SEX']

target = "PROG_JSW"
id_column = "ID"


label_encoder = LabelEncoder()

oai_fnih['P01INJR'] = label_encoder.fit_transform(oai_fnih['P01INJR'])
oai_fnih['P01BP30'] = label_encoder.fit_transform(oai_fnih['P01BP30'])
oai_fnih['P01FAMHR'] = label_encoder.fit_transform(oai_fnih['P01FAMHR'])
oai_fnih['P01FAMKR'] = label_encoder.fit_transform(oai_fnih['P01FAMKR'])
oai_fnih['P01INJL'] = label_encoder.fit_transform(oai_fnih['P01INJL'])
oai_fnih['P02SEX'] = label_encoder.fit_transform(oai_fnih['P02SEX'])

X_all = oai_fnih[features].values

Y_all = oai_fnih[target].astype(int).values

ids = oai_fnih[id_column].values

In [299]:
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score

cv = GroupKFold(n_splits=5)  

cv_scores = []

for train_idx, test_idx in cv.split(X_all, Y_all, groups=ids):
    X_train, X_test = X_all[train_idx], X_all[test_idx]
    Y_train, Y_test = Y_all[train_idx], Y_all[test_idx]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, Y_train)
    knn_preds = knn.predict(X_test)  
    
    score = balanced_accuracy_score(Y_test, knn_preds)
    cv_scores.append(score)


In [ ]:
score = np.mean(cv_scores)
std = np.std(cv_scores)
print(f"CV score: {score:.2f}±{std:.2f}")

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold

rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=5)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_full_preds = np.zeros_like(Y_all, dtype=float)
knn_full_preds = np.zeros_like(Y_all, dtype=float)


rf_fold_scores = []
knn_fold_scores = []

cv_scores = []

for train_idx, test_idx in cv.split(X_all, Y_all):
    X_train, X_test = X_all[train_idx], X_all[test_idx]
    Y_train, Y_test = Y_all[train_idx], Y_all[test_idx]

    rf_model.fit(X_train, Y_train)
    rf_fold_preds = rf_model.predict(X_test)
    rf_full_preds[test_idx] = rf_fold_preds
    rf_fold_scores.append(balanced_accuracy_score(Y_test, rf_fold_preds))

    knn_model.fit(X_train, Y_train)
    knn_fold_preds = knn_model.predict(X_test)
    knn_full_preds[test_idx] = knn_fold_preds
    knn_fold_scores.append(balanced_accuracy_score(Y_test, knn_fold_preds))


rf_balanced_acc = balanced_accuracy_score(Y_all, rf_full_preds)
knn_balanced_acc = balanced_accuracy_score(Y_all, knn_full_preds)


print(f"Random Forest out-of-fold balanced accuracy: {rf_balanced_acc:.2f}")

print(f"kNN out-of-fold balanced accuracy: {knn_balanced_acc:.2f}")

In [ ]:
from scipy.stats import wilcoxon

result = wilcoxon(rf_fold_scores, knn_fold_scores, alternative='greater')

p_value = result.pvalue

print(f"P-value: {p_value:.4f}")


In [ ]:
alpha = 0.05
if p_value < alpha:
    print("There is a big difference between the models.")
else:
    print("There is a not much of a big difference between the models.")

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from scipy.stats import wilcoxon

def encode_categorical_variables(df):
    """
    Encode categorical variables in the dataframe to numerical values.
    """
    categorical_columns = df.select_dtypes(include=['object']).columns
    label_encoders = {}
    
    for column in categorical_columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column].astype(str))
        label_encoders[column] = le
    
    return df, label_encoders

def prepare_data(df):
    """
    Prepare X_all, Y_all, and ids arrays from the dataframe.
    """
    # Encode categorical variables
    df_encoded, _ = encode_categorical_variables(df)
    
    # Separate features and target
    Y_all = df_encoded['PROG_JSW'].values
    X_all = df_encoded.drop(['PROG_JSW', 'ID'], axis=1)
    ids = df_encoded['ID'].values
    
    return X_all, Y_all, ids


In [10]:
oai_fnih.dropna(inplace=True)
X_all, Y_all, ids = prepare_data(oai_fnih)
    
# Convert to numpy arrays
X_all = np.array(X_all)
Y_all = np.array(Y_all)

In [11]:

def patient_level_prediction(y_pred_proba, patient_ids, threshold=0.5):
    """
    Convert knee-level predictions to patient-level predictions.
    A patient is considered positive if either knee shows progression.
    """
    unique_ids = np.unique(patient_ids)
    patient_predictions = []
    
    for pid in unique_ids:
        patient_mask = patient_ids == pid
        knee_probs = y_pred_proba[patient_mask]
        # If any knee has probability > threshold, patient is positive
        patient_pred = 1 if np.any(knee_probs > threshold) else 0
        patient_predictions.append(patient_pred)
    
    return np.array(patient_predictions)

def get_out_of_fold_predictions(X, y, groups, model, n_splits=5):
    """
    Get out-of-fold predictions using GroupKFold cross-validation.
    """
    group_kfold = GroupKFold(n_splits=n_splits)
    oof_predictions = np.zeros(len(y))
    
    for fold, (train_idx, val_idx) in enumerate(group_kfold.split(X, y, groups)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train = y[train_idx]
        
        model.fit(X_train, y_train)
        oof_predictions[val_idx] = model.predict(X_val)
    
    return oof_predictions

def compare_models_balanced_accuracy(X, y, groups):
    """
    Compare Random Forest and kNN models using balanced accuracy
    and statistical testing.
    """
    # Initialize models
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    knn_model = KNeighborsClassifier(n_neighbors=5)
    
    # Get out-of-fold predictions for both models
    rf_predictions = get_out_of_fold_predictions(X, y, groups, rf_model)
    knn_predictions = get_out_of_fold_predictions(X, y, groups, knn_model)
    
    # Calculate balanced accuracy for each model
    rf_ba = balanced_accuracy_score(y, rf_predictions)
    knn_ba = balanced_accuracy_score(y, knn_predictions)
    
    # Prepare data for Wilcoxon test
    # Calculate per-class accuracies for each sample
    n_classes = len(np.unique(y))
    rf_per_sample = np.zeros(len(y))
    knn_per_sample = np.zeros(len(y))
    
    for i in range(n_classes):
        mask = (y == i)
        rf_correct = (rf_predictions == y) & mask
        knn_correct = (knn_predictions == y) & mask
        rf_per_sample[mask] = rf_correct[mask]
        knn_per_sample[mask] = knn_correct[mask]
    
    # Perform Wilcoxon signed-rank test
    statistic, p_value = wilcoxon(rf_per_sample, knn_per_sample)
    
    return {
        'rf_balanced_accuracy': rf_ba,
        'knn_balanced_accuracy': knn_ba,
        'wilcoxon_statistic': statistic,
        'p_value': p_value
    }


In [12]:
results = compare_models_balanced_accuracy(X_all, Y_all, ids)
    
print("Results:")
print(f"Random Forest Balanced Accuracy: {results['rf_balanced_accuracy']:.4f}")
print(f"kNN Balanced Accuracy: {results['knn_balanced_accuracy']:.4f}")
print(f"Wilcoxon test statistic: {results['wilcoxon_statistic']}")
print(f"P-value: {results['p_value']:.4f}")

if results['p_value'] < 0.05:
    print("\nThe difference between models is statistically significant")
else:
    print("\nThe difference between models is not statistically significant")

Results:
Random Forest Balanced Accuracy: 0.7299
kNN Balanced Accuracy: 0.5139
Wilcoxon test statistic: 6975.0
P-value: 0.0000

The difference between models is statistically significant
